In [ ]:
import argparse 
import json 
import logging
import sys 
import time 
from functools import partial 
from pathlib import Path 


import tensorflow as tf 
import nbimporter
import input ; import model_fns ; import predict_fns 
from inputs import * 
from model_fns import * 
from predict_fns import * 


In [ ]:
# this program was designed to funcion with multiple kinds of, but currently 
# but currently, but currently only GPT2 is supported
# Trương trình này được thiết kế để hoạt động với nhiều loại mô hình, nhưng hiện tại chỉ hỗ chợ GPT2
# phần tử đầu tiên trong tuple là hàm mô hình , phần tử thứ 2 là hàm được gọi ki dự đoán 
models = {
    "GPT2": (gpt2_model, gpt2_predict)
}

inputs = {
    "openwebtext": openwebtext, # Standard OpenWebtext input
    "openwebtext_longbiased": openwebtext_longbiased, # OpenWebtext with a bias towards showing more long (>512 tokens) examples
    "openwebtext_long": openwebtext_long, # Openwebtext that only shows long examples
}

# sử dụng thư viện argparse để xử lý accs tham số dòng lệnh 
# argparse tạo một giao diện dòng lệnh cho người dùng nhập các tham số khi chương trình 

# kiểm tra điều kiện để đảm bảo rằng đoạn mã sẽ chỉ chạy khi tệp tin Python được thực thi như 
# một chương trình chính , không phải khi nó được nhập như Module 
if __name__ == "__main__":
    # tạo một parser mới 
    parser = argparse.ArgumentParser()
    # tHÊM CÁC tham số dòng lệnh 
    parser.add_argument('--tpu', type=str) # Tên của TPU để huấn luyện , nếu có 
    parser.add_argument('--model', type=str) # tệp Json chưas các tham số mô hình 
    parser.add_argument('--predict_text',dtype=str) # lấy chuỗi trực tiếp từ tham số 
    parser.add_argument('--top_k', type=int) # Tham số cắt giảm top K cho sinh văn bản 

    # phân tích các tham số được cung cấp 
    args = parser.parse_args()

    # khởi tạo biến để kiểm tra chế độ dự đoán 
    predict_model = False 
    # Kiểm tra nếu tham số predict_file được  cung cấp 
    if args.predict_file is not None: 
        # gán cho biến kiểm tra chế độ dự đoán = True 
        predict_model = True 
        # mở tệp predict_file và  gán kết quả cho f 
        with open(args.predict_file) as f: 
            # đọc file f và gán kết quả cho biến text 
            text = f.read()
        
    # nếu tha số predict_text được cung câos 
    elif args.predict_text is not None: 
        # gán predict_model = TREU 
        predict_model = True 
        # gán biến text = args.predict_text 
        text = args.predict_text 
    # Nếu cả hai tham số predict_file và predict_text đều được cung cấp
    elif args.predict_file is not None and args.predict_text is not None:
        # In ra lỗi và thoát chương trình
        print("ERROR: Specify exactly one of --predict_file and --predict_text!")
        sys.exit()
    

    # Thiết lập ghi nhật ký 
    # Sử dụng path để tạo một đường dẫn là logs 
    # và mkdir để tạo thư mục logs tại đường dẫn path 
    Path('logs').mkdir(exist_ok=True)
    # Thiết lập mức độ chi tiết set_verbosity cho việc ghi nhật ký log là Info 
    tf.logging.set_verbosity(logging.INFO)
    # tạo một danh sách các trình xử lý log 
    handlers = [
        # Trình xử lý ghi log vào tệp, đặt tên tệp log dựa trên mô hình 
        logging.FileHandler(filename='log/{}.log'.format(args.model)),
        # Trình xử llys ghi ra stdout (thường là màn hình console)
        logging.StreamHandler(sys.stdout)
    ]

    # lấy logger của tensorflow 
    logger = logging.getLogger('tensorflow')
    # gán các trình xử lý log đã tạo cho logger 
    logger.handlers = handlers 

    # đọc các tham số của mô hình gán
    # mở bộ tham số của mô hình args.model gán nó cho f các file này được
    # lưu trự dạng tệp json
    with open(args.model, "r") as f :
        # tải các file json từ tệp gán nó cho params 
        params = json.load(f)

    # kiểm tr xem nếu tham số args.tpu tồn tại tức có sử dung 
    if not args.tpu is None : 
        # gán cho giá trị use_tpu trong từ điển params = True 
        params['use_tpu'] = True
    # trường hợp còn lại gán = FALSE
    else: 
        params['use_tpu'] = False 

    # nếu tham số top_k được cung câos 
    if args.top_k is not None: 
        # thiết lập nó tring từ điển params 
        params['top_k'] = args.top_k

    # nếu không có khóa percision trong từ điển params 
    if not 'percision' in params.key():
        # thiết lập nó  mặc định là "float32"
        params["precision"] = "float32" # Doesn't actually do anything since 
        # float32 is the default anyways. Only recognized other dtype is "bfloat16"

    # Nếu không có khóa "iterations" trong params, thiết lập mặc định là 1
    if not "iterations" in params.keys():
        params["iterations"] = 1 # Vì điều này kiểm soát số lượng mẫu được tải trước

    # Ghi thông tin của params ra log
    logger.info(params)

    # Lấy hàm mô hình từ từ điển models dựa trên tên mô hình được cung cấp
    # lấy phần tử đầu tiên của danh sách model được trả về bới khóa 
    # params['model'] trong từ điển model 
    model_fn = models[params["model"]][0]
    # Lấy hàm dự đoán từ từ điển models
    # là lấy phần tử số 2 của danh sách model được trả về bới khóa 
    # params['model'] trong từ điển model 
    predict_fn = models[params["model"]][1]
    # Lấy hàm nhập liệu từ từ điển inputs dựa trên loại đầu vào được cung cấp
    input_fn = inputs[params["input"]]


    # kiểm tra xem mô hình có đang sử dụng TPU và có đang ở chế độ dự đoán không 
    if params["use_tpu"] and not predict_model:
        # giải quyết cụm TPU và chạy các cấu hình 
        tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(args.tpu)

        # chạy các cấu hình , chuyền vào đó các tham số cần thiếu 
        # đường danax thư mục , các cụm giải quyết , lưu trữ checkpoint 
        # và phiên bản cấu hình 
        run_config = tf.contrib.tpu.RunConfig(
            model_dir=params["model_path"],
            cluster=tpu_cluster_resolver,
            save_checkpoints_secs=60*30,
            session_config=tf.ConfigProto(
                # allow_soft_placement=True,
                # log_device_placement=True
                ),
                tpu_config=tf.contrib.tpu.TPUConfig(iterations_per_loop=params["iterations"])
        )

        # Set up network
        # Thiết lập Tpu cho mạng 
        network = tf.contrib.tpu.TPUEstimator(
                model_fn=model_fn,
                use_tpu=True, # chế độ sử dụng TPU 
                # kích thước lô đào tạo
                train_batch_size=params["train_batch_size"], # These are the global sizes, must be divisible by replicas
                # kích thước lô thử nghiệm 
                eval_batch_size=params["eval_batch_size"],
                # kích thước lô cho việc dự đoán
                predict_batch_size=params["predict_batch_size"],
                config=run_config,
                params=params)
    
    # trường hợp không sử dụng TPU ta thiết lập các tham số 
    else:
        # Non TPU setup
        # nếu mô hình không ở chế độ dự đoán
        if not predict_model:
            # gọi khóa batch_size của từ điển params gán nó = giá 
            # trị khóa train_batch_size 
            params["batch_size"] = params["train_batch_size"]
        else:
            # truuwongf hợp còn lại ta cũng gán tương tự 
            params["batch_size"] = params["predict_batch_size"]

            # gọi phương thức encoder của model.gpt2
            from models.gpt2 import encoder
            # mã hóa trước đường dẫn thư mục encoder_path được lưu trữ trong từ điển params 
            # kết quả gán cho biến enc 
            enc = encoder.get_encoder(params["encoder_path"])
            # sử dụng bộ tham số đã học được để mã hóa văn bản cần dự đoán
            tokens = enc.encode(text)
            # gán cho khóa của text_len trong từ điển params = giá trị len(tokens)
            params["text_len"] = len(tokens)
            # kiểm tra xem gái trị text len > 1024
            if params["text_len"] > 1024:
                # nếu có ta gán lại giá trị anyf  = 1024
                params["text_len"] = 1024

        # chạy các cấu hình 
        run_config = tf.estimator.RunConfig(
            model_dir=params["model_path"],
            session_config=tf.ConfigProto(
                # log_device_placement=True,
                # allow_soft_placement=True
            ),
        )

        # sử dụng Estimator là một API cao cấp trong TensorFlow giúp tạo và quản lý các 
        # mô hình học máy. Nó xử lý các vòng lặp huấn luyện, đánh giá, và dự đoán, 
        # giúp việc triển khai mô hình trở nên dễ dàng và hiệu quả hơn. 
        network = tf.estimator.Estimator(
            model_fn=model_fn,
            config=run_config,
            params=params)
        
    # nếu mô hình đang ở chế độ dự đoán predict_model = True 
    if predict_model : 
        # ghi vào nhật ký log một thông tin quá trình dự đoán đang được tạo ra 
        logger.info('Generating predictions...')
        # truyền các tham số vào predict_fn  để thực hiện dự đoán 
        # với các tham số cần thiết 
        predict_fn(network, text, params)
        # thoát khỏi chương trình 
        sys.exit()

    # Xây dựng một vòng lặp vô tận sử dụng cho việc train và thẩm định 
    while True :
        # lấy thời gian hiện tại ghi vào biến start 
        start = time.time()
        # huấn luyện mạng sử dụng hàm part ở đây để có thể tái sử dụng giá trị input_fn có sẵn 
        # cho các mục đích khác tránh viêcj phải viết lại 
        network.train(
                input_fn=partial(input_fn, eval=False),
                # số bước đào tạo 
                steps=params["train_steps"])


        # lấy thời gián hiện tại sau ghi huấn luyện xong lưu kết 
        # quả cho biến end 
        end = time.time()
        # ghi vào log một thông tin thời gian mất để huấn luyện vòng lặp 
        logger.info("\nTrain loop took {:.2f}s\n".format(end-start))

        # xác thực dữ liệu evaluate 
        eval_result = network.evaluate(
           input_fn=partial(input_fn, eval=True),
           # số bước lặp 
           steps=params["eval_steps"])

        # tương tự như trên ghi nhật ký thông tin 
        logger.info("\nEval Results: {}\n".format(str(eval_result)))

        # kiểm tra 1 điều kiện nếu đúng thì thông báo xong và thoát khỏi vòng lặp  
        if network.get_variable_value("global_step") > params["max_steps"]:
            logger.info("Done!")
            break